<a href="https://colab.research.google.com/github/Ezhil531/Sentiment-analysis-using-product-review-data/blob/main/majorproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random

"""Importing Dataset"""

dataset = pd.read_csv('/content/drive/MyDrive/dataset')

# Taking only useful data from dataset
dataset = dataset.iloc[:,[17,16,14,11]].values

"""Taking Care of missing data"""

# Discarding observations having missing data
data = pd.DataFrame(dataset)
dataset = data.dropna(axis = 0,how = 'any')
X = dataset.iloc[:, :-1].values

# Storing rating saperately to use it later 
rating = X[:,-1]

y = dataset.iloc[:, -1].values

# Encoding True/False value as 1/0
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

"""Visualizing Dataset"""

dataset.head(-1)

print(X)
print(y)

"""Cleaning the Dataset"""

import re
import nltk
# Downloading stopwords which does not having much significance
nltk.download('stopwords')
from nltk.corpus import stopwords

# Stemmer will reduce words in their root form
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
all_stopwords = stopwords.words('english')

# Removing some stopwords which have significance effect in building this model
rem = ['not', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', 
       "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', 
       "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't", 'don', "don't", 
       'just', 'too', 'very', 'no', 'nor', 'only', 'own', 'same', 'again', 'against', 'but',]
for s in rem:
  all_stopwords.remove(s)

def find_clean_text(temp):
  # Removing all characters other than alphabet
  temp = re.sub('[^a-zA-Z]', ' ', temp)
  temp = temp.lower()
  temp = temp.split()
  temp = [ps.stem(word) for word in temp if not word in set(all_stopwords)]
  temp = ' '.join(temp)
  return temp

corpus = []
for i in range(X.shape[0]):
  # Concatanating both title and detailed review
  temp = X[i][0] + ' ' + X[i][1]
  temp = find_clean_text(temp)
  corpus.append(temp)

"""Creating the Bag of Words model"""

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 3000)
X = cv.fit_transform(corpus).toarray()

# Adding rating in the matrix of feature X
rating = rating.reshape(rating.shape[0],1)
X = np.append(X,rating,axis=1)

"""Splitting the dataset into the Training set and Test set"""

# Splitting dataset into test set and train set which have equal percentage of data with both positive and negative review
#This is done as precautionary measure considering that very small number (approx 1300 out of 34000) data have negative review.
pos_x = []
pos_y = []
neg_x = []
neg_y = []
for i in range(X.shape[0]):
  if y[i]==1:
    pos_x.append(X[i])
    pos_y.append(y[i])
  else:
    neg_x.append(X[i])
    neg_y.append(y[i])

from sklearn.model_selection import train_test_split
X_train1, X_test1, y_train1, y_test1 = train_test_split(pos_x, pos_y, test_size = 0.20)
X_train, X_test, y_train, y_test = train_test_split(neg_x, neg_y, test_size = 0.20)

for i in range(len(X_train1)):
  X_train.append(X_train1[i])
  y_train.append(y_train1[i])
for i in range(len(X_test1)):
  X_test.append(X_test1[i])
  y_test.append(y_test1[i])
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

"""Training the Multinomial Naive Bayes model on the Training set"""

from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

"""Making the Confusion Matrix"""

from sklearn.metrics import confusion_matrix, accuracy_score

print('Result on training set :')
print('Confusion matrix :')
print(confusion_matrix(y_train,classifier.predict(X_train)))
print('accuracy : ',accuracy_score(y_train, classifier.predict(X_train)))

print('Result on test set :')
y_pred = classifier.predict(X_test)
print('Confusion matrix :')
print(confusion_matrix(y_test, y_pred))
print('accuracy',accuracy_score(y_test, y_pred))

"""Making Prediction on some reviews from test set"""

random.seed(13)
index = random.randrange(dataset.shape[0])
print('Title :',dataset[0][index],'\nReview :',dataset[1][index],'\nRating :',dataset[2][index])
print("True value :", dataset[3][index])
print("Prediction :",classifier.predict(np.append(cv.transform([find_clean_text(dataset[0][index]+' '+dataset[1][index])]).toarray(),[[dataset[3][index]]],axis=1)))

index = random.randrange(dataset.shape[0])
print('Title :',dataset[0][index],'\nReview :',dataset[1][index],'\nRating :',dataset[2][index])
print("True value :", dataset[3][index])
print("Prediction :",classifier.predict(np.append(cv.transform([find_clean_text(dataset[0][index]+' '+dataset[1][index])]).toarray(),[[dataset[3][index]]],axis=1)))

index = random.randrange(dataset.shape[0])
print('Title :',dataset[0][index],'\nReview :',dataset[1][index],'\nRating :',dataset[2][index])
print("True value :", dataset[3][index])
print("Prediction :",classifier.predict(np.append(cv.transform([find_clean_text(dataset[0][index]+' '+dataset[1][index])]).toarray(),[[dataset[3][index]]],axis=1)))

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


[['Kindle'
  'This product so far has not disappointed. My children love to use it and I like the ability to monitor control what content they see with ease.'
  5.0]
 ['very fast'
  'great for beginner or experienced person. Bought as a gift and she loves it'
  5.0]
 ['Beginner tablet for our 9 year old son.'
  'Inexpensive tablet for him to use and learn on, step up from the NABI. He was thrilled with it, learn how to Skype on it already...'
  5.0]
 ...
 ['Love it'
  'Simply the best to watch tv series and movies. It works even better if you are an Amazon Prime subscriber, with access to a many free goodies.'
  5.0]
 ['Try it, you will like it'
  'I was looking for ways to cut cost from a raising cable bill and a friend suggested I try the Amazon Fire. At first I didn���t know if this was something I could do. Once I was able to maneuver through the process, I love it.'
  4.0]
 ['Great little device'
  'I enjoy my kindle tv, it beats paying for cable every month ������'
  4.0]]
[1 1 1